### Algoritmo Genético para o Problema de Localização de Centros de Distribuição
Este notebook implementa um algoritmo genético para resolver o problema de localização de centros de distribuição (Facility Location Problem). O objetivo é selecionar o melhor centro de distribuição que minimize os custos totais de transporte enquanto atende a demanda de todos os clientes, considerando as distâncias reais entre os locais.


#### Importando Bibliotecas e Carregando os Dados
Nesta seção, importamos as bibliotecas necessárias e carregamos os dados dos clientes, centros de distribuição e custos unitários de transporte.

In [3]:
import numpy as np
import pandas as pd
import random
from geopy.distance import geodesic

# Carregar os dados dos arquivos anexados
clients_df = pd.read_excel('input/clients.xlsx')
facilities_df = pd.read_excel('input/facilities.xlsx')
transport_cost_df = pd.read_excel('input/unit_transport_cost.xlsx')

#### Pré-processamento dos Dados
Extraímos informações essenciais dos dados carregados, como nomes, demandas dos clientes, capacidades dos centros de distribuição e as coordenadas geográficas.


In [2]:
# Extraindo os nomes das cidades clientes e dos centros de distribuição
client_names = clients_df['Nome'].tolist()
facility_names = facilities_df['Nome'].tolist()

# Extraindo as demandas dos clientes e as capacidades dos centros
client_demands = clients_df['Demanda'].values
facility_capacities = facilities_df['Capacidade'].values

# Extraindo as coordenadas dos clientes e centros
client_coords = list(zip(clients_df['Latitude'], clients_df['Longitude']))
facility_coords = list(zip(facilities_df['Latitude'], facilities_df['Longitude']))

# Extraindo os custos unitários de transporte por km
unit_transport_costs = transport_cost_df[client_names].values

# Atualizando os parâmetros do algoritmo com base nos dados fornecidos
num_facilities = len(facility_names)
num_clients = len(client_names)

#### Definição dos Parâmetros do Algoritmo Genético
Definimos os parâmetros para o algoritmo genético, incluindo o tamanho da população, o número de gerações, e as taxas de cruzamento e mutação.

In [3]:
# Parâmetros do Algoritmo Genético
population_size = 50  # Tamanho da população
num_generations = 300  # Número de gerações
crossover_rate = 0.8  # Taxa de cruzamento
mutation_rate = 0.1  # Taxa de mutação

#### Função de Aptidão com Distâncias Reais
A função de aptidão é responsável por avaliar o custo total de uma solução, penalizando soluções inviáveis, como a seleção de mais de um centro de distribuição ou a ultrapassagem da capacidade. Utiliza distâncias reais entre clientes e centros de distribuição.

In [4]:
def calculate_fitness(solution):
    total_cost = 0
    capacity_used = np.zeros(num_facilities)
    for client_idx, demand in enumerate(client_demands):
        # Calculando a distância real para cada centro de distribuição disponível
        distances = [geodesic(facility_coords[facility_idx], client_coords[client_idx]).km
                     for facility_idx in range(num_facilities)]
        # Encontrando o centro de distribuição com menor custo total (distância * custo unitário)
        serving_facility_idx = np.argmin(np.array(distances) * unit_transport_costs[:, client_idx] * solution)
        total_cost += distances[serving_facility_idx] * unit_transport_costs[serving_facility_idx, client_idx] * demand
        capacity_used[serving_facility_idx] += demand
    
    # Garantir que apenas um centro de distribuição seja selecionado
    if np.sum(solution) != 1:
        return float('inf')  # Penalidade infinita se mais de um centro for selecionado

    # Penalizar soluções que excedem a capacidade de algum centro
    penalty = np.sum((capacity_used - facility_capacities) * (capacity_used > facility_capacities))
    if penalty > 0:
        return float('inf')  # Penalidade infinita se a capacidade for excedida

    return total_cost

#### Inicialização da População
A população inicial é gerada aleatoriamente, garantindo que cada solução inicial selecione pelo menos um centro de distribuição.

In [5]:
def initialize_population():
    population = []
    for _ in range(population_size):
        solution = np.random.choice([0, 1], size=num_facilities)
        while np.sum(solution) == 0:  # Assegura que pelo menos um centro seja selecionado
            solution = np.random.choice([0, 1], size=num_facilities)
        population.append(solution)
    return population

#### Função de Seleção por Torneio
Seleciona os melhores indivíduos da população para gerar a próxima geração com base na sua aptidão.

In [6]:
def selection(population, fitness_values):
    tournament_size = 3
    selected = []
    for _ in range(population_size):
        participants = random.sample(list(zip(population, fitness_values)), tournament_size)
        winner = min(participants, key=lambda x: x[1])
        selected.append(winner[0])
    return selected

#### Operadores Genéticos: Cruzamento e Mutação
Estas funções aplicam operações de cruzamento e mutação para gerar novas soluções.

In [7]:
def crossover(parent1, parent2):
    if random.random() < crossover_rate:
        point = random.randint(1, num_facilities - 1)
        child1 = np.concatenate((parent1[:point], parent2[point:]))
        child2 = np.concatenate((parent2[:point], parent1[point:]))
        return child1, child2
    return parent1, parent2

def mutate(solution):
    for i in range(num_facilities):
        if random.random() < mutation_rate:
            solution[i] = 1 - solution[i]  # Inverte o gene
    return solution

#### Execução do Algoritmo Genético
Esta célula executa o algoritmo genético por um número especificado de gerações para encontrar a melhor solução.

In [8]:
def run_genetic_algorithm():
    population = initialize_population()
    for generation in range(num_generations):
        fitness_values = [calculate_fitness(ind) for ind in population]
        population = selection(population, fitness_values)
        next_generation = []
        for i in range(0, population_size, 2):
            parent1 = population[i]
            parent2 = population[i + 1]
            child1, child2 = crossover(parent1, parent2)
            next_generation.append(mutate(child1))
            next_generation.append(mutate(child2))
        population = next_generation
    
    # Melhor solução encontrada
    fitness_values = [calculate_fitness(ind) for ind in population]
    best_solution = population[np.argmin(fitness_values)]
    best_fitness = min(fitness_values)
    return best_solution, best_fitness

# Execução do algoritmo ajustado
best_solution, best_fitness = run_genetic_algorithm()

# Coordenadas do centro de distribuição selecionado
selected_facility_idx = np.array(np.where(best_solution == 1)[0])
selected_facility_coords = np.array(facility_coords)[selected_facility_idx]


# Exibindo o resultado final com nome do centro e detalhes dos custos
selected_facility_name = np.array(facility_names)[selected_facility_idx]

print(f"Centro de Distribuição Selecionado: {selected_facility_name}")
print(f"Melhor solução: {best_solution}")
print(f"Custo total: {best_fitness}")

Centro de Distribuição Selecionado: ['São Paulo']
Melhor solução: [1 0 0 0 0]
Custo total: 44679279.56648781


#### Output em Dataframe
Esta celula monta o dataframe de saida, demonstrando os destinos, demanda, distância, origem e custo.

In [10]:
# Calculando as distâncias geográficas reais para cada cliente em relação aos centros selecionados
selected_facility_indices = np.where(best_solution == 1)[0]
distances = []
costs = []
origins = []

for client_idx, client_coord in enumerate(client_coords):
    client_distances = []
    client_costs = []
    client_origins = []
    
    for facility_idx in selected_facility_indices:
        distance = geodesic(facility_coords[facility_idx], client_coord).km
        cost = distance * unit_transport_costs[facility_idx, client_idx] * client_demands[client_idx]
        
        client_distances.append(distance)
        client_costs.append(cost)
        client_origins.append(facility_names[facility_idx])
    
    # Escolhendo a origem com o menor custo para cada cliente
    min_cost_idx = np.argmin(client_costs)
    distances.append(client_distances[min_cost_idx])
    costs.append(client_costs[min_cost_idx])
    origins.append(client_origins[min_cost_idx])

# Criando o DataFrame final com distâncias geográficas reais e custos
final_df = pd.DataFrame({
    'Cliente': client_names,
    'Demanda': client_demands,
    'Distância do Centro Selecionado (km)': np.round(distances, 2),
    'Origem': origins,
    'Custo': np.round(costs, 2)
})

final_df.to_excel('output/resultados.xlsx', index=False)

final_df.head()

,Cliente,Demanda,Distância do Centro Selecionado (km),Origem,Custo
0,Porto Alegre,553,850.62,São Paulo,82396.18
1,Brasília,2767,870.84,São Paulo,977140.17
2,Curitiba,4675,338.83,São Paulo,697391.64
3,Recife,7743,2125.42,São Paulo,5326427.36
4,Goiânia,8680,808.54,São Paulo,1934469.81
